In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0330 18:27:56.275328933    2108 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0330 18:27:59.095647821    2108 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [59]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [60]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [61]:
# # ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']
ran_operators = [300]

In [62]:
pbar = tqdm()

0it [00:00, ?it/s]

### debug AC

In [77]:
itp_id = 273
analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables

In [78]:
rt_day = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
cached parquet empty, will try a fresh query
found parquet
cached parquet empty, will try a fresh query
found parquet
cached parquet empty, will try a fresh query


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


AssertionError: vehicle positions trip ids not in schedule

In [79]:
utils.get_vehicle_positions(273, analysis_date)

found parquet


,calitp_itp_id,calitp_url_number,header_timestamp,vehicle_timestamp,entity_id,vehicle_id,trip_id,vehicle_longitude,vehicle_latitude
0,273,0,2022-03-23 20:30:03,2022-03-23 20:29:54,None,1581,1135040,-121.373085,38.554043
1,273,0,2022-03-23 20:30:03,2022-03-23 20:29:45,None,1580,218040,-121.427100,38.552593
2,273,0,2022-03-23 20:30:03,2022-03-23 20:29:55,None,2821,None,-121.382866,38.603497
3,273,0,2022-03-23 20:30:03,2022-03-23 20:29:36,None,1578,None,-121.382740,38.606770
4,273,0,2022-03-23 20:30:03,2022-03-23 20:29:56,None,1576,1800040,-121.280960,38.678210
...,...,...,...,...,...,...,...,...,...
230039,273,0,2022-03-23 15:08:29,2022-03-23 15:08:12,None,2817,1036040,-121.267160,38.679726
230040,273,0,2022-03-23 15:08:29,2022-03-23 15:08:25,None,2816,None,-121.469930,38.567432
230041,273,0,2022-03-23 15:08:29,2022-03-23 15:08:08,None,2815,1825040,-121.306206,38.576920
230042,273,0,2022-03-23 15:08:29,2022-03-23 15:08:08,None,1603,None,-121.383750,38.595890


In [88]:
analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables

In [89]:
utils.get_trips(273, analysis_date)

found parquet
cached parquet empty, will try a fresh query


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_id,route_short_name


In [84]:
trips = (tbl.views.gtfs_schedule_fact_daily_trips()
# >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
>> filter(_.calitp_itp_id == itp_id)
>> filter(_.service_date == analysis_date)
>> filter(_.is_in_service == True)
>> select(_.trip_key, _.service_date)
>> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
>> select(_.calitp_itp_id, _.calitp_url_number, _.service_date,
          _.trip_key, _.trip_id, _.route_id, _.direction_id,
          _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
        )

In [97]:
analysis_date

datetime.date(2022, 3, 23)

In [112]:
trips = (tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.calitp_itp_id == itp_id)
    >> filter(_.service_date > '2022-02-23', _.service_date < '2022-03-31')
    >> filter(_.is_in_service == True)
    >> count(_.service_date)
    >> collect()
        )

trips

,service_date,n
0,2022-02-28,2560
1,2022-03-01,2560
2,2022-02-24,2560
3,2022-02-25,2560
4,2022-03-02,2560
5,2022-02-26,1452
6,2022-02-27,1158


In [ ]:
def get_trips(itp_id, analysis_date, force_clear=False):
    ''' 
    itp_id: an itp_id (string or integer)
    analysis_date: datetime.date
    
    Interim function for getting complete trips data for a single operator on a single date of interest.
    To be replaced as RT views are implemented...
    
    Updated to include route_short_name from routes
    '''
    
    date_str = analysis_date.strftime('%Y-%m-%d')
    filename = f'trips_{itp_id}_{date_str}.parquet'
    path = check_cached(filename)
    if path and not force_clear:
        print('found parquet')
        cached = pd.read_parquet(path)
        if not cached.empty:
            return cached
        else:
            print('cached parquet empty, will try a fresh query')
    trips = (tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> select(_.trip_key, _.service_date)
        >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
        >> select(_.calitp_itp_id, _.calitp_url_number, _.service_date,
                  _.trip_key, _.trip_id, _.route_id, _.direction_id,
                  _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
        >> collect()
        >> distinct(_.trip_id, _keep_all=True)
        >> inner_join(_, get_routes(itp_id, analysis_date), on = 'route_id')
            )
    if not path or force_clear:
        trips.to_parquet(f'{GCS_FILE_PATH}cached_views/{filename}')
    return trips

In [65]:
rt_day.set_filter(start_time='15:00', end_time='19:00')
m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_names': None, 'shape_ids': None, 'direction_id': None, 'direction': None}
60_shp_0_ empty!
195_shp_0_ empty!
46_shp_0_ empty!
231_shp_0_ empty!
36_shp_0_ empty!
88_shp_1_ empty!
26_shp_0_ empty!
89_shp_0_ empty!
58_shp_1_ empty!
33_shp_1_ empty!
negative speed for shape 210_shp, dropping
210_shp_0_ empty!
negative speed for shape 186_shp, dropping
186_shp_0_ empty!
185_shp_1_ empty!
198_shp_1_ empty!
speed above 80 for shape 199_shp, dropping
199_shp_0_ empty!
194_shp_1_ empty!
20_shp_0_ empty!
19_shp_0_ empty!
209_shp_1_ empty!
213_shp_1_ empty!
43_shp_1_ empty!
227_shp_1_ empty!
negative speed for shape 227_shp, dropping
232_shp_1_ empty!
17_shp_1_ empty!
speed above 80 for shape 17_shp, dropping
50_shp_1_ empty!
25_shp_1_ empty!
48_shp_0_ empty!
233_shp_0_ empty!
234_shp_0_ empty!
stop_speeds shape: (0, 18), shape_id: 37_shp, direction_id: 1
stop speeds gdf is empty!
37_shp_0_ empty!
38_shp_1_

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:604: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [67]:
# m

#### Run all operators

In [69]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [74]:
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1]]

In [76]:
for agency in air_joined.calitp_itp_id.unique():
# for agency in [300]:
    not_ran_operators = []
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables
    day = str(analysis_date.day).zfill(2)
    month = str(analysis_date.month).zfill(2)
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        delay_to_parquet = rt_day.stop_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet['arrival_time'] = delay_to_parquet.arrival_time.map(lambda x: x.isoformat())
        delay_to_parquet['actual_time'] = delay_to_parquet.actual_time.map(lambda x: x.isoformat())
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}stop_delay_views/',
                                        f'{agency}_{month}_{day}'
                                        )
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
        shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
                                                 f'{utils.GCS_FILE_PATH}segment_speed_views/',
                                                f'{agency}_{month}_{day}_pm_peak'
                                                )
        rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_{month}_{day}_pm_peak.parquet') ## early step towards scaling
        # m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
        # rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}endpoint_delay_views/',
                                        f'{agency}_{month}_{day}_pm_peak'
                                        )
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 380...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 380
vehicle positions trip ids not in schedule
already ran: 300
already ran: 182
already ran: 243
already ran: 295
already ran: 170
already ran: 45
already ran: 75
already ran: 4
already ran: 310
calculating for agency: 314...
rt failed for agency 314
no vehicle positions data found for 2022-03-23
already ran: 247
already ran: 315
already ran: 301
already ran: 290
already ran: 167
already ran: 294
already ran: 336
already ran: 194
already ran: 246
already ran: 218
already ran: 282
already ran: 350
already ran: 127
already ran: 110
calculating for agency: 203...
found parquet
found parquet
found parquet
found parquet


/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


5 scheduled trips out of 1411 have no shape, dropping
could not generate delays for trip t17D-sl2-p26-r99
Columns must be same length as key
could not generate delays for trip t185-sl2-p26-r99
Columns must be same length as key
could not generate delays for trip t184-sl2-p26-r98
Columns must be same length as key
could not generate delays for trip t17C-sl2-p26-r98
Columns must be same length as key
could not generate delays for trip t4EB-sl2-p85-r9C
Columns must be same length as key
could not generate delays for trip t4EF-sl2-p85-r9C
Columns must be same length as key
could not generate delays for trip t4ED-sl2-p85-r9C
Columns must be same length as key
could not generate delays for trip t4EC-sl2-p84-r9C
Columns must be same length as key
could not generate delays for trip t4EE-sl2-p84-r9C
Columns must be same length as key
could not generate delays for trip t504-sl2-p89-r9E
Columns must be same length as key
could not generate delays for trip t791-sl2-p89-r9E
Columns must be same len

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_names': None, 'shape_ids': None, 'direction_id': None, 'direction': None}
60_shp_0_ empty!
195_shp_0_ empty!
46_shp_0_ empty!
231_shp_0_ empty!
36_shp_0_ empty!
88_shp_1_ empty!
26_shp_0_ empty!
89_shp_0_ empty!
58_shp_1_ empty!
33_shp_1_ empty!
negative speed for shape 210_shp, dropping
210_shp_0_ empty!
negative speed for shape 186_shp, dropping
186_shp_0_ empty!
185_shp_1_ empty!
198_shp_1_ empty!
speed above 80 for shape 199_shp, dropping
199_shp_0_ empty!
194_shp_1_ empty!
20_shp_0_ empty!
19_shp_0_ empty!
209_shp_1_ empty!
213_shp_1_ empty!
43_shp_1_ empty!
227_shp_1_ empty!
negative speed for shape 227_shp, dropping
232_shp_1_ empty!
17_shp_1_ empty!
speed above 80 for shape 17_shp, dropping
50_shp_1_ empty!
25_shp_1_ empty!
48_shp_0_ empty!
233_shp_0_ empty!
234_shp_0_ empty!
stop_speeds shape: (0, 18), shape_id: 37_shp, direction_id: 1
stop speeds gdf is empty!
37_shp_0_ empty!
38_shp_1_

...

rt failed for agency 372
routelines must not be empty
already ran: 235
calculating for agency: 360...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 360
vehicle positions trip ids not in schedule
already ran: 269
already ran: 30
calculating for agency: 159...
found parquet
found parquet
found parquet
found parquet


/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


could not generate delays for trip t_1057260_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057289_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1439234_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057374_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057367_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057372_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1547393_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057376_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057373_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1547394_b_25985_tn_0
'NoneType' object has no attribut

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_names': None, 'shape_ids': None, 'direction_id': None, 'direction': None}
rt failed for agency 159
'GeoDataFrame' object has no attribute 'trip_id'
already ran: 135
already ran: 293
already ran: 208
already ran: 126
already ran: 361
already ran: 188
already ran: 148
calculating for agency: 346...
rt failed for agency 346
no vehicle positions data found for 2022-03-23
already ran: 226
already ran: 278
already ran: 259


### Ongoing issues
* 358 Union City too many trips without shape?

In [ ]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')